In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

%load_ext autoreload
%autoreload 2

# making all pandas columns visable with display command
pd.set_option('display.max_columns', 500)

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Activation, Flatten, Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam

In [3]:
X = np.load("X_2d.npy")
y = np.load("y_2d.npy")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 41-75-3

In [5]:
# We first need to define the sequence of dependencies (the computational graph)
x = Input(shape=(20,32))
# h = Conv2D(32, (7, 7), strides=(2, 2))(x)
# h = Activation('relu')(x)
h = Flatten()(x)
h = Dense(41)(h)
h = Dense(75)(h)
# h = Dense(40)(h)
# h = Dense(40)(h)
h = Dropout(0.3)(h)
p = Activation('softmax')(h)

# Now that we have defined how to find p from x, we can create a 
# model simply by saying what is input and what is output
model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 32)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 640)               0         
_________________________________________________________________
dense (Dense)                (None, 41)                26281     
_________________________________________________________________
dense_1 (Dense)              (None, 75)                3150      
_________________________________________________________________
dropout (Dropout)            (None, 75)                0         
_________________________________________________________________
activation (Activation)      (None, 75)                0         
Total params: 29,431
Trainable params: 29,431
Non-trainable params: 0
_________________________________________________________

In [6]:
history = model.fit(X_train, y_train, batch_size=64, epochs=200, validation_split=0.04)

Train on 2010 samples, validate on 84 samples
Epoch 1/200
2010/2010 [==============================] - 0s 238us/sample - loss: 901883.7586 - accuracy: 0.0000e+00 - val_loss: 661337.6161 - val_accuracy: 0.0000e+00
Epoch 2/200
2010/2010 [==============================] - 0s 33us/sample - loss: 855082.5091 - accuracy: 0.0000e+00 - val_loss: 622786.7173 - val_accuracy: 0.0000e+00
Epoch 3/200
2010/2010 [==============================] - 0s 31us/sample - loss: 809311.2088 - accuracy: 0.0000e+00 - val_loss: 584618.5357 - val_accuracy: 0.0000e+00
Epoch 4/200
2010/2010 [==============================] - 0s 29us/sample - loss: 764758.1901 - accuracy: 0.0000e+00 - val_loss: 546596.6488 - val_accuracy: 0.0000e+00
Epoch 5/200
2010/2010 [==============================] - 0s 32us/sample - loss: 714363.1907 - accuracy: 0.0000e+00 - val_loss: 509213.0565 - val_accuracy: 0.0000e+00
Epoch 6/200
2010/2010 [==============================] - 0s 31us/sample - loss: 672611.5201 - accuracy: 0.0000e+00 - val_lo

Epoch 52/200
2010/2010 [==============================] - 0s 31us/sample - loss: 24275.2507 - accuracy: 0.2433 - val_loss: 4281.8164 - val_accuracy: 0.2500
Epoch 53/200
2010/2010 [==============================] - 0s 32us/sample - loss: 22998.6944 - accuracy: 0.2423 - val_loss: 3933.2622 - val_accuracy: 0.2738
Epoch 54/200
2010/2010 [==============================] - 0s 33us/sample - loss: 21845.3967 - accuracy: 0.2572 - val_loss: 3906.3402 - val_accuracy: 0.2976
Epoch 55/200
2010/2010 [==============================] - 0s 31us/sample - loss: 20996.7544 - accuracy: 0.2577 - val_loss: 3819.1771 - val_accuracy: 0.3095
Epoch 56/200
2010/2010 [==============================] - 0s 32us/sample - loss: 20610.4995 - accuracy: 0.2632 - val_loss: 3808.1704 - val_accuracy: 0.2738
Epoch 57/200
2010/2010 [==============================] - 0s 32us/sample - loss: 19345.2751 - accuracy: 0.2512 - val_loss: 3781.1878 - val_accuracy: 0.2619
Epoch 58/200
2010/2010 [==============================] - 0s 31u

Epoch 105/200
2010/2010 [==============================] - 0s 42us/sample - loss: 3938.4277 - accuracy: 0.3204 - val_loss: 2032.9836 - val_accuracy: 0.4167
Epoch 106/200
2010/2010 [==============================] - 0s 46us/sample - loss: 4015.8877 - accuracy: 0.3333 - val_loss: 1993.8284 - val_accuracy: 0.4286
Epoch 107/200
2010/2010 [==============================] - 0s 40us/sample - loss: 3886.0114 - accuracy: 0.3244 - val_loss: 2006.7693 - val_accuracy: 0.4167
Epoch 108/200
2010/2010 [==============================] - 0s 40us/sample - loss: 3881.7126 - accuracy: 0.3184 - val_loss: 2057.4808 - val_accuracy: 0.4405
Epoch 109/200
2010/2010 [==============================] - 0s 34us/sample - loss: 3768.1803 - accuracy: 0.3254 - val_loss: 2017.0686 - val_accuracy: 0.3452
Epoch 110/200
2010/2010 [==============================] - 0s 34us/sample - loss: 3962.1318 - accuracy: 0.3075 - val_loss: 2009.6238 - val_accuracy: 0.2976
Epoch 111/200
2010/2010 [==============================] - 0s 35

2010/2010 [==============================] - 0s 32us/sample - loss: 3172.4748 - accuracy: 0.3388 - val_loss: 1470.2195 - val_accuracy: 0.4762
Epoch 158/200
2010/2010 [==============================] - 0s 32us/sample - loss: 3337.0338 - accuracy: 0.3318 - val_loss: 1490.4058 - val_accuracy: 0.4524
Epoch 159/200
2010/2010 [==============================] - 0s 32us/sample - loss: 3284.1668 - accuracy: 0.3259 - val_loss: 1447.4248 - val_accuracy: 0.4048
Epoch 160/200
2010/2010 [==============================] - 0s 31us/sample - loss: 3083.0495 - accuracy: 0.3373 - val_loss: 1491.2640 - val_accuracy: 0.4643
Epoch 161/200
2010/2010 [==============================] - 0s 33us/sample - loss: 3138.2043 - accuracy: 0.3338 - val_loss: 1441.4579 - val_accuracy: 0.4643
Epoch 162/200
2010/2010 [==============================] - 0s 32us/sample - loss: 2924.2715 - accuracy: 0.3174 - val_loss: 1447.9412 - val_accuracy: 0.3929
Epoch 163/200
2010/2010 [==============================] - 0s 32us/sample - lo

## LSTM  8-41-75-75-75-3

In [7]:
# We first need to define the sequence of dependencies (the computational graph)
x = Input(shape=(20,32))
# h = Conv2D(32, (7, 7), strides=(2, 2))(x)
# h = Activation('relu')(x)
# h = Flatten()(x)
h = LSTM(8)(x)
h = Dense(41)(h)
h = Dense(75)(h)
h = Dense(75)(h)
h = Dense(75)(h)
# h = Dense(40)(h)
# h = Dense(40)(h)
h = Dropout(0.3)(h)
p = Activation('softmax')(h)

# Now that we have defined how to find p from x, we can create a 
# model simply by saying what is input and what is output
model = Model(inputs=x, outputs=p)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.00001), metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20, 32)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 8)                 1312      
_________________________________________________________________
dense_2 (Dense)              (None, 41)                369       
_________________________________________________________________
dense_3 (Dense)              (None, 75)                3150      
_________________________________________________________________
dense_4 (Dense)              (None, 75)                5700      
_________________________________________________________________
dense_5 (Dense)              (None, 75)                5700      
_________________________________________________________________
dropout_1 (Dropout)          (None, 75)                0   

In [8]:
history = model.fit(X_train, y_train, batch_size=64, epochs=200, validation_split=0.04)

Train on 2010 samples, validate on 84 samples
Epoch 1/200
2010/2010 [==============================] - 2s 1ms/sample - loss: 4.1659 - accuracy: 9.9502e-04 - val_loss: 4.1466 - val_accuracy: 0.0000e+00
Epoch 2/200
2010/2010 [==============================] - 0s 120us/sample - loss: 4.1340 - accuracy: 4.9751e-04 - val_loss: 4.1129 - val_accuracy: 0.0000e+00
Epoch 3/200
2010/2010 [==============================] - 0s 118us/sample - loss: 4.0997 - accuracy: 0.0184 - val_loss: 4.0785 - val_accuracy: 0.0000e+00
Epoch 4/200
2010/2010 [==============================] - 0s 123us/sample - loss: 4.0650 - accuracy: 0.0557 - val_loss: 4.0439 - val_accuracy: 0.0000e+00
Epoch 5/200
2010/2010 [==============================] - 0s 126us/sample - loss: 4.0375 - accuracy: 0.0746 - val_loss: 4.0088 - val_accuracy: 0.2024
Epoch 6/200
2010/2010 [==============================] - 0s 130us/sample - loss: 3.9978 - accuracy: 0.2627 - val_loss: 3.9729 - val_accuracy: 0.4643
Epoch 7/200
2010/2010 [===============

2010/2010 [==============================] - 0s 141us/sample - loss: 2.2430 - accuracy: 0.3945 - val_loss: 1.7601 - val_accuracy: 0.4643
Epoch 56/200
2010/2010 [==============================] - ETA: 0s - loss: 2.1974 - accuracy: 0.39 - 0s 129us/sample - loss: 2.1906 - accuracy: 0.3985 - val_loss: 1.7411 - val_accuracy: 0.4643
Epoch 57/200
2010/2010 [==============================] - 0s 129us/sample - loss: 2.2053 - accuracy: 0.3925 - val_loss: 1.7215 - val_accuracy: 0.4643
Epoch 58/200
2010/2010 [==============================] - 0s 126us/sample - loss: 2.2040 - accuracy: 0.3940 - val_loss: 1.7052 - val_accuracy: 0.4643
Epoch 59/200
2010/2010 [==============================] - 0s 131us/sample - loss: 2.2472 - accuracy: 0.3940 - val_loss: 1.6896 - val_accuracy: 0.4643
Epoch 60/200
2010/2010 [==============================] - 0s 135us/sample - loss: 2.1883 - accuracy: 0.3975 - val_loss: 1.6759 - val_accuracy: 0.4643
Epoch 61/200
2010/2010 [==============================] - 0s 127us/samp

2010/2010 [==============================] - 0s 125us/sample - loss: 2.1275 - accuracy: 0.4035 - val_loss: 1.4423 - val_accuracy: 0.4643
Epoch 110/200
2010/2010 [==============================] - 0s 124us/sample - loss: 2.1108 - accuracy: 0.3985 - val_loss: 1.4398 - val_accuracy: 0.4643
Epoch 111/200
2010/2010 [==============================] - 0s 141us/sample - loss: 2.1220 - accuracy: 0.3985 - val_loss: 1.4384 - val_accuracy: 0.4643
Epoch 112/200
2010/2010 [==============================] - 0s 124us/sample - loss: 2.1325 - accuracy: 0.4005 - val_loss: 1.4387 - val_accuracy: 0.4643
Epoch 113/200
2010/2010 [==============================] - 0s 133us/sample - loss: 2.1192 - accuracy: 0.3910 - val_loss: 1.4373 - val_accuracy: 0.4643
Epoch 114/200
2010/2010 [==============================] - 0s 129us/sample - loss: 2.0814 - accuracy: 0.3975 - val_loss: 1.4352 - val_accuracy: 0.4643
Epoch 115/200
2010/2010 [==============================] - 0s 135us/sample - loss: 2.0950 - accuracy: 0.4104

2010/2010 [==============================] - 0s 135us/sample - loss: 2.0336 - accuracy: 0.4060 - val_loss: 1.3043 - val_accuracy: 0.4643
Epoch 164/200
2010/2010 [==============================] - 0s 134us/sample - loss: 2.0397 - accuracy: 0.3900 - val_loss: 1.3022 - val_accuracy: 0.4643
Epoch 165/200
2010/2010 [==============================] - 0s 136us/sample - loss: 1.9900 - accuracy: 0.4095 - val_loss: 1.2989 - val_accuracy: 0.4643
Epoch 166/200
2010/2010 [==============================] - 0s 134us/sample - loss: 2.0762 - accuracy: 0.4005 - val_loss: 1.2990 - val_accuracy: 0.4643
Epoch 167/200
2010/2010 [==============================] - 0s 130us/sample - loss: 2.0358 - accuracy: 0.3955 - val_loss: 1.2963 - val_accuracy: 0.4643
Epoch 168/200
2010/2010 [==============================] - 0s 133us/sample - loss: 2.0256 - accuracy: 0.3990 - val_loss: 1.2945 - val_accuracy: 0.4643
Epoch 169/200
2010/2010 [==============================] - 0s 133us/sample - loss: 2.0296 - accuracy: 0.4179